In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/mm22d016/Multi-Task')
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import RobertaModel 
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from dataset import Downstream_Dataset
import torch
import torch.nn as nn
from copy import deepcopy
from torch.utils.data import DataLoader

/home/mm22d016/miniconda3/envs/TransPolymer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fingerprint = torch.empty(3716, 768)

In [3]:
class DownstreamRegression(nn.Module):
    def __init__(self, drop_rate=0.1):
        super(DownstreamRegression, self).__init__()
        self.PretrainedModel = deepcopy(PretrainedModel)
        self.PretrainedModel.resize_token_embeddings(len(tokenizer))
        
        self.Regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(self.PretrainedModel.config.hidden_size, self.PretrainedModel.config.hidden_size),
            nn.SiLU(),
            nn.Linear(self.PretrainedModel.config.hidden_size, 1)
        )

    def forward(self, input_ids, attention_mask,step):
        outputs = self.PretrainedModel(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.last_hidden_state[:, 0, :] #fingerprint
        fingerprint[step] = logits
        output = self.Regressor(logits)
        return None
    
def test(model, train_dataloader,device):
    
    model.eval()
    with torch.no_grad():
        for step, batch in enumerate(train_dataloader):
            print(f'Smiles: {step+1}')
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            prop = batch["prop"].to(device).float()
            outputs = model(input_ids, attention_mask,step)

In [4]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = pd.read_csv('../data/multiTask.csv')
data

,smiles,Eea,Egb,Egc,Ei,Eea_fingerprints,Egb_fingerprints,Egc_fingerprints
0,*CC(*)C,0.4343,7.6997,6.5196,6.1850,[-1.18903264e-01 5.12989938e-01 -6.88638449e-...,[-1.60840774e+00 -5.40946066e-01 2.52460182e-...,[-1.60317779e+00 2.88874637e-02 -5.70353210e-...
1,*CC(*)F,0.8740,8.3099,7.6225,7.6332,[ 1.07876800e-01 -2.75215089e-01 -1.02893257e+...,[-1.78167951e+00 -1.06529927e+00 3.16144347e-...,[-1.50895000e+00 -7.13937402e-01 2.64854163e-...
2,*CC(*)(F)F,1.1415,-99.0000,7.1498,8.1531,[-5.12820005e-01 2.40226462e-02 -1.61661911e+...,[-1.92897761e+00 -8.38245273e-01 4.49982695e-...,[-1.93003619e+00 -3.69191498e-01 9.08003077e-...
3,*C(F)C(*)(F)F,1.5240,6.8451,7.6787,8.5986,[-5.03343940e-01 -2.49886945e-01 -1.39350855e+...,[-1.56505358e+00 -1.12169015e+00 3.28355968e-...,[-2.0849538 -0.5660355 0.28373533 -0.263375...
4,*CCC(F)(F)C(*)(F)F,0.4489,9.1316,8.6882,9.0178,[ 3.02336842e-01 1.76058039e-01 -8.18257868e-...,[-1.13705683e+00 -5.86708605e-01 6.71963990e-...,[-1.53351128e+00 -5.52727163e-01 5.50605297e-...
...,...,...,...,...,...,...,...,...
3711,*Nc1c([2H])c([2H])c(*)c([2H])c1[2H],-99.0000,-99.0000,3.3666,-99.0000,[ 1.91717818e-01 -4.00347322e-01 -4.25487190e-...,[-0.7404429 0.26716265 -0.7208934 0.964733...,[ 0.6999478 0.24567693 -0.1644021 -0.642044...
3712,*CCCCCC[N+](C)(C)CCC[N+](*)(C)C,-99.0000,-99.0000,0.2161,-99.0000,[ 1.25339293e+00 4.54728156e-01 -1.58412233e-...,[ 8.76836836e-01 -7.22541213e-01 1.14970064e+...,[ 9.76258039e-01 -4.23593581e-01 -1.71856150e-...
3713,*CCCCCCCC[N+](C)(C)CCCCCC[N+](*)(C)C,-99.0000,-99.0000,0.1032,-99.0000,[ 1.44612479e+00 5.86507976e-01 -8.86027291e-...,[ 1.36068881e+00 -5.33318698e-01 1.00794160e+...,[ 9.27366436e-01 -3.92986119e-01 -3.17045122e-...
3714,*CCCCCCCCCCCCCCCC[N+](C)(C)CCCCCC[N+](*)(C)C,-99.0000,-99.0000,0.1771,-99.0000,[ 1.50361729e+00 5.09277046e-01 -7.71710128e-...,[ 1.35130703e+00 -4.95626330e-01 1.00366926e+...,[ 9.04213190e-01 -3.14821810e-01 -2.49084175e-...


In [5]:
saved_state = torch.load('../models/Ei/Ei_best_model.pt')
vocab_sup = pd.read_csv('../data/vocab/vocab_sup_PE_II.csv', header=None).values.flatten().tolist()

PretrainedModel = RobertaModel.from_pretrained('../ckpt/pretrain.pt')
tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=411)
tokenizer.add_tokens(vocab_sup)
dataset = Downstream_Dataset(data, tokenizer, 411)

model = DownstreamRegression(drop_rate=0.1).to(device)
model = model.double()
model.load_state_dict(saved_state['model'])
torch.cuda.empty_cache()

train_dataloader = DataLoader(dataset, 1, shuffle=False, num_workers=2)

Some weights of the model checkpoint at ../ckpt/pretrain.pt were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../ckpt/pretrain.pt and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer clas

In [ ]:
print("hello")
test(model, train_dataloader, device)

fingerprint = np.round(fingerprint.detach().cpu().numpy(), decimals=9)

hello
Smiles: 1
Smiles: 2
Smiles: 3
Smiles: 4
Smiles: 5
Smiles: 6
Smiles: 7
Smiles: 8
Smiles: 9
Smiles: 10
Smiles: 11
Smiles: 12
Smiles: 13
Smiles: 14
Smiles: 15
Smiles: 16
Smiles: 17
Smiles: 18
Smiles: 19
Smiles: 20
Smiles: 21
Smiles: 22
Smiles: 23
Smiles: 24
Smiles: 25
Smiles: 26
Smiles: 27
Smiles: 28
Smiles: 29
Smiles: 30
Smiles: 31
Smiles: 32
Smiles: 33
Smiles: 34
Smiles: 35
Smiles: 36
Smiles: 37
Smiles: 38
Smiles: 39
Smiles: 40
Smiles: 41
Smiles: 42
Smiles: 43
Smiles: 44
Smiles: 45
Smiles: 46
Smiles: 47
Smiles: 48
Smiles: 49
Smiles: 50
Smiles: 51
Smiles: 52
Smiles: 53
Smiles: 54
Smiles: 55
Smiles: 56
Smiles: 57
Smiles: 58
Smiles: 59
Smiles: 60
Smiles: 61
Smiles: 62
Smiles: 63
Smiles: 64
Smiles: 65
Smiles: 66
Smiles: 67
Smiles: 68
Smiles: 69
Smiles: 70
Smiles: 71
Smiles: 72
Smiles: 73
Smiles: 74
Smiles: 75
Smiles: 76
Smiles: 77
Smiles: 78
Smiles: 79
Smiles: 80
Smiles: 81
Smiles: 82
Smiles: 83
Smiles: 84
Smiles: 85
Smiles: 86
Smiles: 87
Smiles: 88
Smiles: 89
Smiles: 90
Smiles: 91
Sm

Smiles: 706
Smiles: 707
Smiles: 708
Smiles: 709
Smiles: 710
Smiles: 711
Smiles: 712
Smiles: 713
Smiles: 714
Smiles: 715
Smiles: 716
Smiles: 717
Smiles: 718
Smiles: 719
Smiles: 720
Smiles: 721
Smiles: 722
Smiles: 723
Smiles: 724
Smiles: 725
Smiles: 726
Smiles: 727
Smiles: 728
Smiles: 729
Smiles: 730
Smiles: 731
Smiles: 732
Smiles: 733
Smiles: 734
Smiles: 735
Smiles: 736
Smiles: 737
Smiles: 738
Smiles: 739
Smiles: 740
Smiles: 741
Smiles: 742
Smiles: 743
Smiles: 744
Smiles: 745
Smiles: 746
Smiles: 747
Smiles: 748
Smiles: 749
Smiles: 750
Smiles: 751
Smiles: 752
Smiles: 753
Smiles: 754
Smiles: 755
Smiles: 756
Smiles: 757
Smiles: 758
Smiles: 759
Smiles: 760
Smiles: 761
Smiles: 762
Smiles: 763
Smiles: 764
Smiles: 765
Smiles: 766
Smiles: 767
Smiles: 768
Smiles: 769
Smiles: 770
Smiles: 771
Smiles: 772
Smiles: 773
Smiles: 774
Smiles: 775
Smiles: 776
Smiles: 777
Smiles: 778
Smiles: 779
Smiles: 780
Smiles: 781
Smiles: 782
Smiles: 783
Smiles: 784
Smiles: 785
Smiles: 786
Smiles: 787
Smiles: 788
Smil

Smiles: 1362
Smiles: 1363
Smiles: 1364
Smiles: 1365
Smiles: 1366
Smiles: 1367
Smiles: 1368
Smiles: 1369
Smiles: 1370
Smiles: 1371
Smiles: 1372
Smiles: 1373
Smiles: 1374
Smiles: 1375
Smiles: 1376
Smiles: 1377
Smiles: 1378
Smiles: 1379
Smiles: 1380
Smiles: 1381
Smiles: 1382
Smiles: 1383
Smiles: 1384
Smiles: 1385
Smiles: 1386
Smiles: 1387
Smiles: 1388
Smiles: 1389
Smiles: 1390
Smiles: 1391
Smiles: 1392
Smiles: 1393
Smiles: 1394
Smiles: 1395
Smiles: 1396
Smiles: 1397
Smiles: 1398
Smiles: 1399
Smiles: 1400
Smiles: 1401
Smiles: 1402
Smiles: 1403
Smiles: 1404
Smiles: 1405
Smiles: 1406
Smiles: 1407
Smiles: 1408
Smiles: 1409
Smiles: 1410
Smiles: 1411
Smiles: 1412
Smiles: 1413
Smiles: 1414
Smiles: 1415
Smiles: 1416
Smiles: 1417
Smiles: 1418
Smiles: 1419
Smiles: 1420
Smiles: 1421
Smiles: 1422
Smiles: 1423
Smiles: 1424
Smiles: 1425
Smiles: 1426
Smiles: 1427
Smiles: 1428
Smiles: 1429
Smiles: 1430
Smiles: 1431
Smiles: 1432
Smiles: 1433
Smiles: 1434
Smiles: 1435
Smiles: 1436
Smiles: 1437
Smiles: 1438

In [ ]:
data['Ei_fingerprints'] = [fingerprint[i] for i in range(3716)]
data['Ei_fingerprints'].apply(np.array)
data['Ei_fingerprints']

In [ ]:
data['Ei_fingerprints'][0]

In [ ]:
data.to_csv('../data/multiTask.csv',index=False)